In [ ]:
!pip install contractions nltk

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import pandas as pd
import contractions
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fatma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fatma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fatma\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
df = pd.read_csv('spotifyDataset.csv', engine='python', on_bad_lines='skip')
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


**Preprocessing phase**

In [ ]:
df.isna().sum() #check for na but there is no missing values

artist    0
song      0
link      0
text      0
dtype: int64

In [ ]:
#Convert to lowercase and also we create new column to keep the original data
df["Lyrics"] = df["text"].str.lower()
df["song_name"] = df["song"].str.lower()
df.head()

,artist,song,link,text,Lyrics,song_name
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","look at her face, it's a wonderful face \nand...",ahe's my kind of girl
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","take it easy with me, please \ntouch me gentl...","andante, andante"
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,i'll never know why i had to go \nwhy i had t...,as good as new
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,making somebody happy is a question of give an...,bang
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,making somebody happy is a question of give an...,bang-a-boomerang


In [ ]:
#It converts English contractions into their full forms (e.g., they’re → they are, don’t → do not). because when we delete unneccessary characters the ' will be deleted and the sentence will be wrong so they're will be theyre so this is unacceptable
df["Lyrics"] = df["Lyrics"].apply(
    lambda x: contractions.fix(x)
)
df["song_name"] = df["song_name"].apply(
    lambda x: contractions.fix(x)
)
df.head(9)

,artist,song,link,text,Lyrics,song_name
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","look at her face, it is a wonderful face \nan...",ahe's my kind of girl
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","take it easy with me, please \ntouch me gentl...","andante, andante"
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,i will never know why i had to go \nwhy i had...,as good as new
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,making somebody happy is a question of give an...,bang
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,making somebody happy is a question of give an...,bang-a-boomerang
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ...","well, you hoot and you holler and you make me ...",burning my bridges
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...,down in the street they are all singing and sh...,cassandra
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \nYou're enc...","chiquitita, tell me what is wrong \nyou are e...",chiquitita
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \nCouldn't sle...,i was out with the morning sun \ncould not sl...,crazy world


In [ ]:
# Normalize spaces, keep line breaks
df["Lyrics"] = df["Lyrics"].str.replace(
    r"[ \t]+", " ", regex=True
).str.strip()
df.head(20)


,artist,song,link,text,Lyrics,song_name
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","look at her face, it is a wonderful face \nand...",ahe's my kind of girl
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","take it easy with me, please \ntouch me gently...","andante, andante"
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,i will never know why i had to go \nwhy i had ...,as good as new
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,making somebody happy is a question of give an...,bang
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,making somebody happy is a question of give an...,bang-a-boomerang
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ...","well, you hoot and you holler and you make me ...",burning my bridges
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...,down in the street they are all singing and sh...,cassandra
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \nYou're enc...","chiquitita, tell me what is wrong \nyou are en...",chiquitita
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \nCouldn't sle...,i was out with the morning sun \ncould not sle...,crazy world
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \nI'm sitting all al...,i am waitin' for you baby \ni am sitting all a...,crying over you


In [ ]:
#Correct wrong spelling in song column
corrections = {
    "ahe": "she",
    "hte": "the",
    "thier": "their"
}

for wrong, correct in corrections.items():
    df["song_name"] = df["song_name"].str.replace(
        rf"\b{wrong}\b", correct, regex=True
    )
df.head(2)

,artist,song,link,text,Lyrics,song_name
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","look at her face, it is a wonderful face \nand...",she's my kind of girl
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","take it easy with me, please \ntouch me gently...","andante, andante"


Skip this preprocessing part for now As we generate lyrics so removing stop words isn't efficient but we are going to try with another way or let the model predict the stop words

In [ ]:
# stop_words = set(stopwords.words("english"))
#Manually removing stopwords during preprocessing may lead to the unintended loss of words or phrases that are important for distinguishing between songs, particularly when phrase-level patterns are relevant. Since manual removal permanently alters the text, it can negatively affect feature extraction and limit flexibility in experimentation. Therefore, stopword handling is delegated to the TF-IDF vectorizer, which removes common terms dynamically during feature construction while preserving the original text. This approach reduces noise without discarding potentially informative lexical patterns and is more suitable for song identification tasks.
lemmatizer = WordNetLemmatizer()
vectorizer = TfidfVectorizer(
    stop_words="english",   # correct for song detection
    ngram_range=(1, 3),     # unigrams + bigrams + trigrams
    min_df=2                # remove very rare noise
) #skip this for now

In [ ]:
# Remove stopwords
#Original text remains unchanged. Stopwords are removed only in the feature space.You can:change stopword list.re-vectorize.reuse raw text
x = vectorizer.fit_transform(df["Lyrics"])
vectorizer.get_stop_words() #skip this for now until we try during training process

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'